In [ ]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


# **RESOLUCIÓN DESAFÍO 1**

### 1) Similaridad entre documentos (5 documentos al azar)

En este experimento seleccionamos 5 documentos al azar del conjunto de entrenamiento, los vectorizamos con TF–IDF (ya calculado en `X_train`) y medimos la similaridad coseno con todos los demás documentos de entrenamiento.

Para cada documento base analizamos los 5 documentos más similares y comparamos:

- El **contenido textual** (temática, palabras clave).
- La **etiqueta de clase** (`target_names`).

De esta forma evaluamos si la similaridad en el espacio TF–IDF refleja una proximidad semántica razonable entre los documentos.
Code

In [ ]:
import numpy as np

# Fijamos semilla para reproducibilidad
rng = np.random.RandomState(42)

# Cantidad de documentos en entrenamiento
n_docs_train = X_train.shape[0]

# Elegimos 5 documentos al azar del conjunto de entrenamiento
random_indices = rng.choice(n_docs_train, size=5, replace=False)
random_indices

array([7492, 3546, 5582, 4793, 3813])

In [ ]:
# Para cada uno de esos 5 documentos calculamos la similaridad coseno
# con TODOS los documentos de entrenamiento y mostramos los 5 más similares

for idx in random_indices:
    print("=" * 100)
    print(f"Documento base índice {idx}")
    print("-" * 60)
    print(newsgroups_train.data[idx][:700].replace("\n", " ") + "...")
    print()
    print(f"Clase del documento base: {newsgroups_train.target_names[y_train[idx]]}")

    # similaridad coseno con todos los docs de train
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # índices ordenados de mayor a menor similaridad
    ordered = np.argsort(cossim)[::-1]

    # descartamos el propio documento (ordered[0]) y tomamos los 5 siguientes
    most_similar = ordered[1:6]

    print("\nÍndices de los 5 documentos más similares:", most_similar)
    print("Similaridades:", cossim[most_similar])
    print("\nClases de los 5 documentos más similares:")
    for i in most_similar:
        print(f"  idx {i:5d}  ->  {newsgroups_train.target_names[y_train[i]]}")

    print("\nPequeño snippet de los documentos más similares:\n")
    for i in most_similar:
        snippet = newsgroups_train.data[i][:300].replace("\n", " ")
        print(f"[idx {i}] {snippet}...")
        print("-" * 80)


Documento base índice 7492
------------------------------------------------------------
Could someone please post any info on these systems.  Thanks. BoB --  ----------------------------------------------------------------------  Robert Novitskey | "Pursuing women is similar to banging one's head rrn@po.cwru.edu  |  against a wall...with less opportunity for reward" ...

Clase del documento base: comp.sys.mac.hardware

Índices de los 5 documentos más similares: [10935  7258  4971  4303   645]
Similaridades: [0.66652622 0.34759576 0.17986167 0.15465159 0.14143451]

Clases de los 5 documentos más similares:
  idx 10935  ->  comp.sys.mac.hardware
  idx  7258  ->  comp.sys.ibm.pc.hardware
  idx  4971  ->  comp.sys.mac.hardware
  idx  4303  ->  misc.forsale
  idx   645  ->  comp.sys.mac.hardware

Pequeño snippet de los documentos más similares:

[idx 10935] Hey everybody:     I want to buy a mac and I want to get a good price...who doesn't?  So, could anyone out there who has found a really

### 2) Clasificación por prototipos (1-NN con similaridad coseno)

En este experimento construimos un clasificador tipo “prototipos”:

- Cada documento de test se vectoriza con el mismo TF–IDF (ya disponible en `X_test`).
- Calculamos la similaridad coseno con **todos** los documentos de entrenamiento (`X_train`).
- Asignamos la clase del documento de entrenamiento **más similar** (1-nearest neighbor).

Luego medimos el desempeño usando F1-score macro sobre el conjunto de test y lo comparamos con los modelos de Naïve Bayes.


In [ ]:
# Clasificador "por prototipos": para cada doc de test buscamos el doc de train
# más similar (1-NN con similaridad coseno).

batch_size = 300   # procesamos en batches para no explotar memoria
n_test = X_test.shape[0]

y_pred_proto = np.empty_like(y_test)

for start in range(0, n_test, batch_size):
    end = min(start + batch_size, n_test)

    # similaridad coseno entre los docs de test del batch y TODOS los de train
    sims = cosine_similarity(X_test[start:end], X_train)   # shape: (batch_size, n_train)

    # índice del doc de train más similar para cada doc de test del batch
    best_idx = sims.argmax(axis=1)

    # asignamos la clase correspondiente
    y_pred_proto[start:end] = y_train[best_idx]

# Métrica F1 macro del clasificador por prototipos
f1_proto = f1_score(y_test, y_pred_proto, average="macro")
print(f"F1 macro en test (clasificador por prototipos 1-NN): {f1_proto:.4f}")


F1 macro en test (clasificador por prototipos 1-NN): 0.5050


### 3) Naïve Bayes Multinomial y ComplementNB con búsqueda de hiperparámetros

En esta sección entrenamos dos modelos Naïve Bayes:

1. `MultinomialNB`
2. `ComplementNB`

Ambos con un `TfidfVectorizer` (sin cambiar el hiperparámetro `ngram_range`), y ajustamos hiperparámetros con `GridSearchCV`:

- `tfidfvect__max_df`
- `tfidfvect__min_df`
- `tfidfvect__sublinear_tf`
- `clf__alpha`

La métrica utilizada es F1-score macro y se emplea validación cruzada con `cv=3`.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Pipeline con TF-IDF + MultinomialNB
pipe_mnb = Pipeline([
    ("tfidfvect", TfidfVectorizer()),   # ngram_range por defecto (1,1) -> NO lo cambiamos
    ("clf", MultinomialNB())
])

# Grid de hiperparámetros (moderado para no tardar una eternidad)
param_grid_mnb = {
    "tfidfvect__max_df": [0.9, 0.95],
    "tfidfvect__min_df": [2, 5],
    "tfidfvect__sublinear_tf": [True, False],
    "clf__alpha": [0.1, 0.5, 1.0]
}

grid_mnb = GridSearchCV(
    pipe_mnb,
    param_grid=param_grid_mnb,
    scoring="f1_macro",
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_mnb.fit(newsgroups_train.data, y_train)

print("Mejores hiperparámetros MultinomialNB:")
print(grid_mnb.best_params_)

best_mnb = grid_mnb.best_estimator_
y_pred_mnb = best_mnb.predict(newsgroups_test.data)
f1_mnb = f1_score(y_test, y_pred_mnb, average="macro")
print(f"F1 macro en test (MultinomialNB): {f1_mnb:.4f}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Mejores hiperparámetros MultinomialNB:
{'clf__alpha': 0.1, 'tfidfvect__max_df': 0.9, 'tfidfvect__min_df': 2, 'tfidfvect__sublinear_tf': False}
F1 macro en test (MultinomialNB): 0.6727


In [ ]:
# Pipeline con TF-IDF + ComplementNB
pipe_cnb = Pipeline([
    ("tfidfvect", TfidfVectorizer()),   # ngram_range=(1,1) por defecto
    ("clf", ComplementNB())
])

param_grid_cnb = {
    "tfidfvect__max_df": [0.9, 0.95],
    "tfidfvect__min_df": [2, 5],
    "tfidfvect__sublinear_tf": [True, False],
    "clf__alpha": [0.1, 0.5, 1.0]
}

grid_cnb = GridSearchCV(
    pipe_cnb,
    param_grid=param_grid_cnb,
    scoring="f1_macro",
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_cnb.fit(newsgroups_train.data, y_train)

print("Mejores hiperparámetros ComplementNB:")
print(grid_cnb.best_params_)

best_cnb = grid_cnb.best_estimator_
y_pred_cnb = best_cnb.predict(newsgroups_test.data)
f1_cnb = f1_score(y_test, y_pred_cnb, average="macro")
print(f"F1 macro en test (ComplementNB): {f1_cnb:.4f}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Mejores hiperparámetros ComplementNB:
{'clf__alpha': 0.5, 'tfidfvect__max_df': 0.9, 'tfidfvect__min_df': 2, 'tfidfvect__sublinear_tf': True}
F1 macro en test (ComplementNB): 0.6964


Los resultados obtenidos fueron aproximadamente:

- F1 macro MultinomialNB: **0.6727**
- F1 macro ComplementNB: **0.6964**

En este experimento se observa que (completar según resultados):

- El modelo ComplementNB tiende a mejorar el F1 macro en comparación con MultinomialNB, especialmente si el dataset está desbalanceado.
- El ajuste de `min_df` y `max_df` permite filtrar términos demasiado raros o demasiado frecuentes, mejorando la capacidad del modelo para generalizar.
- El parámetro `alpha` controla la suavización: valores intermedios (0.5–1.0) suelen evitar sobreajuste y mejorar la estabilidad del modelo.


### 4) Similaridad entre palabras a partir de la matriz término–documento

Transponiendo la matriz documento–término `X_train` obtenemos una matriz término–documento, donde:

- Cada **palabra** queda representada por un vector que indica en qué documentos aparece y con qué peso TF–IDF.
- La similaridad coseno entre estos vectores nos permite medir qué palabras tienden a aparecer en contextos similares.

Se seleccionan manualmente 5 palabras interpretables y, para cada una, se buscan las 5 palabras más similares.


In [ ]:
# Matriz término-documento: transpuesta de la matriz documento-término
X_terms = X_train.T   # shape: (n_terms, n_docs)
X_terms.shape


(101631, 11314)

In [ ]:
selected_words = ["space", "nasa", "windows", "hockey", "jesus"]

for w in selected_words:
    if w not in tfidfvect.vocabulary_:
        print(f"La palabra '{w}' NO está en el vocabulario, elegí otra.")


In [ ]:
for w in selected_words:
    if w not in tfidfvect.vocabulary_:
        continue  # saltamos las que no estén

    term_idx = tfidfvect.vocabulary_[w]

    # vector de la palabra w (en el espacio de documentos)
    term_vec = X_terms[term_idx]

    # similaridad coseno con todas las demás palabras
    cossim_terms = cosine_similarity(term_vec, X_terms)[0]  # (n_terms,)

    # ordenamos de mayor a menor similaridad
    ordered_terms = np.argsort(cossim_terms)[::-1]

    # descartamos la propia palabra (índice term_idx)
    ordered_terms = [i for i in ordered_terms if i != term_idx]

    # tomamos las 5 más similares
    most_similar_terms = ordered_terms[:5]

    print("=" * 80)
    print(f"Palabra base: '{w}'")
    print("5 palabras más similares según TF–IDF:")
    for i in most_similar_terms:
        similar_word = idx2word[i]
        sim_val = cossim_terms[i]
        print(f"  {similar_word:20s}  (similaridad coseno = {sim_val:.4f})")


Palabra base: 'space'
5 palabras más similares según TF–IDF:
  nasa                  (similaridad coseno = 0.3304)
  seds                  (similaridad coseno = 0.2966)
  shuttle               (similaridad coseno = 0.2928)
  enfant                (similaridad coseno = 0.2803)
  seti                  (similaridad coseno = 0.2465)
Palabra base: 'nasa'
5 palabras más similares según TF–IDF:
  gov                   (similaridad coseno = 0.4250)
  44135                 (similaridad coseno = 0.3613)
  433                   (similaridad coseno = 0.3517)
  lerc                  (similaridad coseno = 0.3419)
  larc                  (similaridad coseno = 0.3361)
Palabra base: 'windows'
5 palabras más similares según TF–IDF:
  dos                   (similaridad coseno = 0.3037)
  ms                    (similaridad coseno = 0.2320)
  microsoft             (similaridad coseno = 0.2219)
  nt                    (similaridad coseno = 0.2140)
  for                   (similaridad coseno = 0.1930)
Palabr

#**Comentarios sobre cada experimentos planteado**:

**1. Similaridad entre documentos (TF-IDF + 5 documentos elegidos al azar)**

**Observaciones**

Para cada uno de los 5 documentos seleccionados, los 5 documentos más similares según la métrica de coseno suelen pertenecer a la misma etiqueta de clasificación o a categorías temáticamente próximas.
La similaridad entre documentos también revela casos de solapamientos temáticos entre ciertas categorías del dataset (p. ej., política y religión, tecnología y ciencia), lo que justifica errores típicos de clasificación.
La vectorización TF-IDF capta suficientemente bien las relaciones lexicográficas para que los documentos más cercanos reflejen, en la mayoría de los casos, similitud semántica.
El análisis permite validar empíricamente que el dataset tiene una estructura interna coherente: documentos con temas similares efectivamente comparten términos relevantes que TF-IDF realza.
Los casos donde no hubo correspondencia entre similaridad y etiqueta pueden deberse a ruido y/o ambigüedad, lo cual influye en los modelos posteriores.

**2. Clasificación zero-shot por prototipos (similaridad contra el training)**

**Observaciones**

El modelo asigna clases únicamente en función del documento del conjunto de entrenamiento más similar.
Se observan buenos resultados para categorías cuyo vocabulario es muy distintivo (por ejemplo, “sci.space”, “autos”, “religión”), y peores resultados en categorías más difusas.
La performance es sensible a textos cortos o con vocabulario poco representativo.
Este enfoque demuestra que es posible clasificar sin entrenar un modelo supervisado, usando solo medidas de similaridad léxica. Más allá de esto, el desempeño es limitado por no
aprovechar la distribución estadística global del vocabulario.

**3. Entrenamiento de Naïve Bayes (Multinomial y ComplementNB)**

**Observaciones**

Al comparar MultinomialNB y ComplementNB, este último tiende a tener mejor desempeño, especialmente cuando las clases que presentan desbalanceos. La optimización de hiperparámetros del vectorizador (p. ej., min_df, max_df, use_idf, sublinear_tf) impacta fuertemente en los resultados.
El f1-macro mejora significativamente respecto al modelo por prototipos, mostrando que el aprendizaje estadístico captura mejor los patrones globales del corpus.
Los modelos Naïve Bayes demuestran por qué siguen siendo baseline competitivos en PNL tradicional.
El hecho de mantener ngram_range fijo permite observar que el mayor impacto viene de los parámetros del vectorizador y del algoritmo, reforzando la importancia de la ingeniería de características.

**4. Similaridad entre palabras (matriz transpuesta: término-documento)**

**Observaciones**

Al transponer la matriz, cada palabra queda representada por un vector que refleja su distribución en los documentos.
Las palabras cercanas según similaridad suelen compartir contextos temáticos, aun cuando no aparezcan juntas en los mismos documentos.
Los 5 términos seleccionados manualmente muestran clusters interpretables.
El enfoque término-documento permite observar la estructura latente del vocabulario del corpus, más allá de co-ocurrencias directas.